In [76]:
#!g1.1
import json
import os
import torch
from tqdm import tqdm
import scipy.io.wavfile as wav

from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.utils.tensorboard import SummaryWriter

import numpy as np
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import math

In [65]:
#!g1.1
# %pip install pytorch_lightning --upgrade

In [66]:
#!g1.1
from models import ConvFeatureEncoder, SegmentsRepr, SegmentsEncoder, NegativeSampler, SegmentPredictor, FinModel, FinModel1
from utils import ConstrativeLoss, sample_negatives
# from trainer import Trainer
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

In [67]:
#!g1.1
# import wandb

In [68]:
#!g1.1
def spectral_size(wav_len):
    layers = [(10,5,0), (8,4,0), (4,2,0), (4,2,0), (4,2,0)]
    for kernel, stride, padding in layers:
        wav_len = math.floor((wav_len + 2*padding - 1*(kernel-1) - 1)/stride + 1)
    return wav_len

In [77]:
#!g1.1
class Dataset:
    
    def __init__(self, path, segment_path, manifest_path, edges_path = None, 
                 chars = 'segments_chars/', frames = 'secs/'):
        with open(manifest_path, 'r') as json_file:
            manifest = json.load(json_file)
        self.manifest = manifest
#         self.manifest = manifest[:1000]
        self.path = path
        self.segment_path = segment_path
        self.frames = os.path.join(chars, frames)
        self.edges_path = edges_path
    
    def __len__(self):
        return len(self.manifest)
    
    def spectral_size(self, wav_len):
        layers = [(10,5,0), (8,4,0), (4,2,0), (4,2,0), (4,2,0)]
        for kernel, stride, padding in layers:
            wav_len = math.floor((wav_len + 2*padding - 1*(kernel-1) - 1)/stride + 1)
        return wav_len
    
    def __getitem__(self, ind):
#         print(self.manifest[ind])
        
        # Загрузка аудио-сигнала
        
        audio_filepath = self.manifest[ind]['audio_filepath']
        audio_file = os.path.join(self.path, audio_filepath)
        sampling_rate, signal = wav.read(audio_file)
        
        # Обрезать тишину
        filetext_id = self.manifest[ind]['id']+'.txt'
        silero_filepath = audio_filepath.replace('.wav', '.txt')
        silero_file = os.path.join(self.edges_path, silero_filepath)
        
        with open(silero_file, 'r', encoding="cp1251") as file:
            tt = file.read()
        
        start = []
        end = []
        for line in tt.split('\n'):
            if len(line) > 0:
                start.append(eval(line.split()[0]))
                end.append(eval(line.split()[1]))
        start = min(start)
        end = max(end)
        signal = signal[start:end]
        
        # Загрузка разметки
        filetext_id = self.manifest[ind]['id']+'.txt'
        segment_filepath = audio_filepath.replace('.wav', '.txt').replace(filetext_id, '')
        
        segment_file = os.path.join(os.path.join(os.path.join(self.segment_path, segment_filepath), self.frames), filetext_id)

        with open(segment_file, 'r', encoding="cp1251") as file:
            tt = file.read()
        
        boundaries = set()
        mm = [i for i in tt.split('\n') if len(i)>0]
        for i in mm:
            boundaries.add(eval(i.split()[0]))
            boundaries.add(eval(i.split()[1]))
        boundaries = sorted(list(boundaries))
        
        return {'id':filetext_id, 'sample': signal, 'length': len(signal), 
                'boundaries': boundaries, 'length': len(signal), 
                'spectral_size': self.spectral_size(len(signal)),
                }

In [78]:
#!g1.1
def collate_fn(samples):
    
    max_length = max([sample['length'] for sample in samples])
    boundaries = [sample['boundaries'] for sample in samples]
    spectral_sizes = [sample['spectral_size'] for sample in samples]
    samples1 = []
    lengths = []
    samplings = []
    attentions = []
    for sample in samples:
        to_add_l = max_length-sample['length']
        sample1 = list(sample['sample'])+[0]*to_add_l
        samples1.append(torch.Tensor(sample1).unsqueeze(0))
        lengths.append(sample['length'])
        att_norm = torch.ones(size = (1, sample['length']))
        att_add = torch.zeros(size = (1, to_add_l))
        att = torch.cat([att_norm, att_add], dim = -1)
        attentions.append(att)
        
    batch = torch.cat(samples1)
    lengths = torch.Tensor(lengths)
    attention_mask = torch.cat(attentions, dim = 0)
    spectral_size = torch.Tensor(spectral_sizes)
    
    return dict(batch=batch, lengths=lengths, attention_mask=attention_mask, boundaries=boundaries, spectral_size=spectral_size)

In [79]:
#!g1.1


In [80]:
#!g1.1


In [81]:
#!g1.1
train_dataset = Dataset('train/', 'segments_edges_init/', '10hours_train.json', 
                        edges_path = 'silero_edges',
                              chars = 'segments_chars/', frames = 'secs/')
val_dataset = Dataset('train/', 'segments_edges_init/', '10hours_val.json', 
                        edges_path = 'silero_edges',
                              chars = 'segments_chars/', frames = 'secs/')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn = collate_fn)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=8, collate_fn = collate_fn)

In [82]:
#!g1.1


In [98]:
#!g1.1
accumulate_grad_batches = 1

In [99]:
#!g1.1
cfg = {'optimizer': "adam",
'momentum': 0.9,
'learning_rate': 0.0001*accumulate_grad_batches,
'lr_anneal_gamma': 1.0,
'lr_anneal_step': 1000,
# 'epochs': 500,
'grad_clip': 0.5,
'batch_size': 8,

'conv_args': {},
'mask_args': {"segment": "first", "add_one": False},
'segm_enc_args': {},
'segm_predictor_args': {},
'loss_args': {"n_negatives": 1, "loss_args": {"reduction": "mean"}},
'num_epoch': 2}

In [100]:
#!g1.1
class Conf:
    def __init__(self, my_dict):
        for key, value in my_dict.items():
            setattr(self, key, value)
            
config = Conf(cfg)

In [101]:
#!g1.1
import warnings
warnings.filterwarnings("ignore")

In [102]:
#!g1.1

AVAIL_GPUS = min(1, torch.cuda.device_count())

train_dataset = Dataset('train/', 'segments_edges_init/', '10hours_train.json', 
                        edges_path = 'silero_edges',
                              chars = 'segments_chars/', frames = 'secs/')
val_dataset = Dataset('train/', 'segments_edges_init/', '10hours_val.json', 
                        edges_path = 'silero_edges',
                              chars = 'segments_chars/', frames = 'secs/')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn = collate_fn)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=8, collate_fn = collate_fn)

model = FinModel(config)

logger = TensorBoardLogger("tb_logs_pl_r_val_acc_200_edges_train_10_hours_model_segment1", name="my_model")

checkpoint_callback = ModelCheckpoint(
    monitor="val_r_metr",
    dirpath="./",
    filename="golos_model_segment_r_val_acc_200_edges_train_10_hours_model_segment1",
    save_top_k=3,
    mode="max",
)
trainer = Trainer(max_epochs=25,
                  gpus=AVAIL_GPUS,
                  progress_bar_refresh_rate=1,
                  logger=logger,
#                   accumulate_grad_batches=accumulate_grad_batches,
                  gradient_clip_val=2, 
                  callbacks=[checkpoint_callback])


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [103]:
#!g1.1
%tensorboard --logdir ./tb_logs_pl_r_val_acc_200_edges_train_10_hours_model_segment1

Launching TensorBoard...


In [104]:
#!g1.1


In [105]:
#!g1.1
trainer.fit(model, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params
---------------------------------------------------------
0 | conv_encoder      | ConvFeatureEncoder | 1.3 M 
1 | frame_predictor   | NegativeSampler    | 0     
2 | segment_mean      | SegmentsRepr       | 0     
3 | attention_calc    | AttentionCalc      | 0     
4 | segment_encoder   | SegmentsEncoder    | 131 K 
5 | segment_recurrent | SegmentPredictor   | 78.5 K
6 | segment_predictor | NegativeSampler    | 0     
7 | metr              | RMetrics           | 0     
---------------------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
6.102     Total estimated model params size (MB)


optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0.0005
)



In [106]:
#!g1.1


In [107]:
#!g1.1


In [108]:
#!g1.1


In [111]:
#!g1.1


In [112]:
#!g1.1

AVAIL_GPUS = min(1, torch.cuda.device_count())

train_dataset = Dataset('train/', 'segments_edges_init/', '10hours_train.json', 
                        edges_path = 'silero_edges',
                              chars = 'segments_chars/', frames = 'secs/')
val_dataset = Dataset('train/', 'segments_edges_init/', '10hours_val.json', 
                        edges_path = 'silero_edges',
                              chars = 'segments_chars/', frames = 'secs/')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn = collate_fn)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=8, collate_fn = collate_fn)

model = FinModel1(config)

logger = TensorBoardLogger("tb_logs_pl_r_val_acc_200_edges_train_10_hours_peak_detection", name="my_model")

checkpoint_callback = ModelCheckpoint(
    monitor="val_r_metr",
    dirpath="./",
    filename="golos_model_segment_r_val_acc_200_edges_train_10_hours_peak_detection",
    save_top_k=3,
    mode="max",
)
trainer = Trainer(max_epochs=25,
                  gpus=AVAIL_GPUS,
                  progress_bar_refresh_rate=1,
                  logger=logger,
#                   accumulate_grad_batches=accumulate_grad_batches,
                  gradient_clip_val=2, 
                  callbacks=[checkpoint_callback])


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [113]:
#!g1.1
%tensorboard --logdir ./tb_logs_pl_r_val_acc_200_edges_train_10_hours_peak_detection

Launching TensorBoard...


In [116]:
#!g1.1
trainer.fit(model, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type               | Params
-------------------------------------------------------
0 | conv_encoder    | ConvFeatureEncoder | 1.3 M 
1 | frame_predictor | NegativeSampler    | 0     
2 | segment_mean    | SegmentsRepr       | 0     
3 | attention_calc  | AttentionCalc      | 0     
4 | metr            | RMetrics           | 0     
5 | unsup_loss      | UnsupLoss          | 0     
6 | bounds          | GetBound           | 0     
-------------------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.261     Total estimated model params size (MB)


optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0.0005
)



In [ ]:
#!g1.1


In [ ]:
#!g1.1


In [ ]:
#!g1.1
# Transformers

In [ ]:
#!g1.1


In [45]:
#!g1.1
# %pip install --upgrade transformers
# %pip install datasets
# %pip install huggingface_hub

In [46]:
#!g1.1
# %pip install --upgrade pip

In [47]:
#!g1.1
import json
import os
import torch
from tqdm import tqdm
import scipy.io.wavfile as wav

from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import torchaudio

In [48]:
#!g1.1
import transformers
import json
from torch.utils.data import Dataset, DataLoader, Sampler
import os

In [49]:
#!g1.1
from models import ConvFeatureEncoder, SegmentsRepr, SegmentsEncoder, NegativeSampler, SegmentPredictor, FinModel, FinModel1
from utils import ConstrativeLoss, sample_negatives

In [50]:
#!g1.1
from model_transformers import SegmentTransformer
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

In [51]:
#!g1.1
transformers.__version__

'4.14.0'

In [52]:
#!g1.1
class Dataset:
    
    def __init__(self, path, segment_path, manifest_path, edges_path = None, 
                 chars = 'segments_chars/', frames = 'secs/'):
        with open(manifest_path, 'r') as json_file:
            manifest = json.load(json_file)
        self.manifest = manifest
#         self.manifest = manifest[:1000]
        self.path = path
        self.segment_path = segment_path
        self.frames = os.path.join(chars, frames)
        self.edges_path = edges_path
    
    def __len__(self):
        return len(self.manifest)
    
    def __getitem__(self, ind):
#         print(self.manifest[ind])
        
        # Загрузка аудио-сигнала
        
        audio_filepath = self.manifest[ind]['audio_filepath']
        audio_file = os.path.join(self.path, audio_filepath)
        sampling_rate, signal = wav.read(audio_file)
        
        # Обрезать тишину
        filetext_id = self.manifest[ind]['id']+'.txt'
        silero_filepath = audio_filepath.replace('.wav', '.txt')
        silero_file = os.path.join(self.edges_path, silero_filepath)
        
        with open(silero_file, 'r', encoding="cp1251") as file:
            tt = file.read()
        
        start = []
        end = []
        for line in tt.split('\n'):
            if len(line) > 0:
                start.append(eval(line.split()[0]))
                end.append(eval(line.split()[1]))
        start = min(start)
        end = max(end)
        signal = signal[start:end]
        
        # Загрузка разметки
        filetext_id = self.manifest[ind]['id']+'.txt'
        segment_filepath = audio_filepath.replace('.wav', '.txt').replace(filetext_id, '')
        
        segment_file = os.path.join(os.path.join(os.path.join(self.segment_path, segment_filepath), self.frames), filetext_id)

        with open(segment_file, 'r', encoding="cp1251") as file:
            tt = file.read()
        
        boundaries = set()
        mm = [i for i in tt.split('\n') if len(i)>0]
        for i in mm:
            boundaries.add(eval(i.split()[0]))
            boundaries.add(eval(i.split()[1]))
        boundaries = sorted(list(boundaries))
        
        return {'id':filetext_id, 'sample': signal, 'length': len(signal), 'boundaries': boundaries}

In [53]:
#!g1.1
def collate_fn(samples):
    
    max_length = max([sample['length'] for sample in samples])
    boundaries = [sample['boundaries'] for sample in samples]
    samples1 = []
    lengths = []
    samplings = []
    attentions = []
    for sample in samples:
        to_add_l = max_length-sample['length']
        sample1 = list(sample['sample'])+[0]*to_add_l
        samples1.append(torch.Tensor(sample1).unsqueeze(0))
        lengths.append(sample['length'])
        att_norm = torch.ones(size = (1, sample['length']))
        att_add = torch.zeros(size = (1, to_add_l))
        att = torch.cat([att_norm, att_add], dim = -1)
        attentions.append(att)
        
    batch = torch.cat(samples1)
    lengths = torch.Tensor(lengths)
    attention_mask = torch.cat(attentions, dim = 0)
    
    return dict(batch=batch, lengths=lengths, attention_mask=attention_mask, boundaries=boundaries)

In [54]:
#!g1.1


In [55]:
#!g1.1
train_dataset = Dataset('train/', 'segments_edges_init/', '10hours_train.json', 
                        edges_path = 'silero_edges',
                              chars = 'segments_chars/', frames = 'secs/')
val_dataset = Dataset('train/', 'segments_edges_init/', '10hours_val.json', 
                        edges_path = 'silero_edges',
                              chars = 'segments_chars/', frames = 'secs/')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn = collate_fn)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=8, collate_fn = collate_fn)

In [56]:
#!g1.1


In [60]:
#!g1.1
import warnings
warnings.filterwarnings("ignore")
from collections import OrderedDict

In [61]:
#!g1.1
AVAIL_GPUS = min(1, torch.cuda.device_count())

train_dataset = Dataset('train/', 'segments_edges_init/', '10hours_train.json', 
                        edges_path = 'silero_edges',
                              chars = 'segments_chars/', frames = 'secs/')
val_dataset = Dataset('train/', 'segments_edges_init/', '10hours_val.json', 
                        edges_path = 'silero_edges',
                              chars = 'segments_chars/', frames = 'secs/')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn = collate_fn)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=8, collate_fn = collate_fn)

model = SegmentTransformer(config)

logger = TensorBoardLogger("tb_logs_pl_transformers_acc_10_ep_500_r_val_edges_train_10_hours_new_loss", name="my_model")

checkpoint_callback = ModelCheckpoint(
    monitor="val_r_metr",
    dirpath="./",
    filename="golos_model_segment_r_val_transformers_acc_10_ep_500_r_val_edges_train_10_hours_new_loss",
    save_top_k=3,
    mode="max",
)
trainer = Trainer(max_epochs=25,
                  gpus=AVAIL_GPUS,
                  progress_bar_refresh_rate=1,
                  logger=logger,
#                   accumulate_grad_batches=accumulate_grad_batches,
                  gradient_clip_val=2, 
                  callbacks=[checkpoint_callback])


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ModelForSegmentation: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2ModelForSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ModelForSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ModelForSegmentation were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.project_back.weight', 'wav2vec2.project_back.bias', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

In [62]:
#!g1.1
%tensorboard --logdir ./tb_logs_pl_transformers_acc_10_ep_500_r_val_edges_train_10_hours_new_loss

Launching TensorBoard...


In [63]:
#!g1.1
trainer.fit(model, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                         | Params
--------------------------------------------------------------
0 | wav2vec_segm | Wav2Vec2ModelForSegmentation | 94.5 M
--------------------------------------------------------------
94.5 M    Trainable params
0         Non-trainable params
94.5 M    Total params
378.013   Total estimated model params size (MB)


optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0.0005
)



In [ ]:
#!g1.1


In [ ]:
#!g1.1


In [ ]:
#!g1.1
